In [8]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.insert(1, '../../')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
import torch
from src.models.ecg_step_module import EcgEmbed
from src.basic.dx_and_feat import Feature, zero_vec

N = 64 # batch_size
batched_ecg = torch.rand(N, 12, 5000)
batched_feat = zero_vec(Feature, [N])
batched_input = (batched_ecg, batched_feat)
all_mid_output = {}
embed = EcgEmbed(all_mid_output)(batched_input)
print(all_mid_output['EcgEmbed']['embed'].shape)

torch.Size([64, 240])


In [10]:
from src.models.ecg_step_module import BlockModule
# all_mid_output['BlockModule'].clear()
block_module = BlockModule(all_mid_output)
block_module(batched_input)
print(all_mid_output['BlockModule']['LPR_imp'])
print(all_mid_output['BlockModule']['LQRS_imp'])

torch.Size([64, 262])
torch.Size([64, 1])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (64x262 and 263x128)